In [1]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from transformers.optimization import AdamW, get_linear_schedule_with_warmup
import torch
import pynvml
import psutil

Using TensorFlow backend.


In [2]:
#注意，DeBERTa开始，最大输入文本就没有限制了，很NB
def temp_collate_fn(batch):
    max_len = max([len(f["input_id"]) for f in batch])
    input_id = [f['input_id'] + [0] * (max_len - len(f['input_id'])) for f in batch]
    input_mask = [[1] * len(f['input_id']) + [0] * (max_len - len(f['input_id'])) for f in batch]
    tags = [f['tags'] for f in batch]
    
    input_id = torch.tensor(input_id)
    input_mask = torch.tensor(input_mask)
    tags = torch.tensor(tags)
    
    output = (input_id, input_mask, tags)
    return output

In [ ]:
def evaluate(model,feature_data,device,tags:str):    #考虑分词的
    preds = []
    dataloader = DataLoader(feature_data, batch_size=2, shuffle=False, collate_fn=temp_collate_fn, drop_last=True)
    model.zero_grad()
    for batch in dataloader:
        model.eval()
        inputs = {'input_id': batch[0].to(device),
                  'input_mask': batch[1].to(device),
              }
        with torch.no_grad():
            output = model(**inputs)     #产出一个list[list],大长度就是所有文档或句子，里面的长度是token_num
            preds.extend(output[0])
    
    fenzi=0
    fenmu=0
    for num in range(len(preds)):        
        y_true=feature_data[num]['tags']
        y_pred=preds[num].argmax(dim=-1)
        if y_true==y_pred:
            fenzi += 1
    fenmu=len(preds)
    p=fenzi/fenmu
    
    print(tags+"数据准确率为:",p,"数据总数:",fenmu,"判断对的数目:",fenzi)
    return p

In [ ]:
#这里都按照enti_softmax的格式改了，没有crf
def train(model, train_features, dev_features,train_batch_size:int, num_epoch, learning_rate,device):
    def finetune(features, optimizer):
        best_score = -1
        train_dataloader = DataLoader(features, batch_size=train_batch_size, shuffle=True, collate_fn=temp_collate_fn, drop_last=True)
        total_steps = int(len(train_dataloader) * num_epoch )
        warmup_steps = int(total_steps * 0.06)
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)
        num_steps=0
        for epoch in range(int(num_epoch)):
            model.zero_grad()
            for step, batch in enumerate(train_dataloader):
                model.train()
                inputs = {'input_id': batch[0].to(device),
                      'input_mask': batch[1].to(device),
                      'tags':batch[2].to(device),
                      }
                outputs = model(**inputs)
                loss = outputs[1] 
                loss.backward()
                #print(step,loss)
                #if loss>10 or torch.isnan(loss)==True:
                #    print(step,loss)
                if step%50==0:
                    print("已读取的数据数：",step*2)
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                scheduler.step()
                model.zero_grad()
                num_steps += 1
                if (step + 1) == len(train_dataloader) - 1 :
                    print("第",epoch,"轮loss为:",loss)
                    dev_F1 = evaluate(model, dev_features,device, tags="dev")
                    if dev_F1 > best_score:
                        best_score = dev_F1
                        torch.save(model.state_dict(),'huifu_model.pth' )
        return num_steps
    
    new_layer = ["FNN"]
    optimizer_grouped_parameters = [
        {"params": [p for n, p in model.named_parameters() if not any(nd in n for nd in new_layer)], "weight_decay":0.01 },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in new_layer)], "lr": learning_rate},
        ]
    
    optimizer = AdamW(optimizer_grouped_parameters, lr=1.5e-5)               #这里的lr是预训练的学习率              
    model.zero_grad()
    finetune(train_features, optimizer)

In [1]:
89/98

0.9081632653061225